# Setup

In [1]:
from dataclasses import dataclass

In [2]:
@dataclass
class Env:
    env:tuple = tuple()
    
    def extend(self,bindings:dict):
        return Env((bindings,) + self.env)
    
    def extend_from_vars_vals(self,vars,vals):
        bindings = dict(zip(vars,vals))
        return self.extend(bindings)
    
    @property
    def current_frame(self):
        return self.env[0]
    
    @property
    def enclosed_env(self):
        return Env(self.env[1:])
    
    def is_empty(self):
        return self.env == ()
    
    def lookup(self,var):
        if self.is_empty():
            raise NameError(f"{var} is undefined")
        elif var not in self.current_frame:
            return self.enclosed_env.lookup(var)
        else:
            return self.current_frame[var]
        
    def __str__(self) -> str:
        return str(tuple(tuple(env.keys()) for env in self.env))

In [3]:
Exp_Val = bool|int|float

@dataclass
class Const:
    val:Exp_Val
    def __iter__(self):
        return iter(("Const",self.val))

@dataclass
class Prim:
    op:str
    exps:tuple
    def __iter__(self):
        return iter(('Prim',self.op,*self.exps))

@dataclass
class Let:
    var:str
    exp:None
    body:None
    def __iter__(self):
        return iter(("Let",self.var,self.exp,self.body))

@dataclass
class Var:
    var:str
    def __iter__(self):
        return iter(("Var",self.var))

# Problem

In [10]:
def lang_int_interp(exp,env:Env = None):
    from operator import sub,add,mul,truediv
    prim_op = {
        '-':sub,
        '+':add,
        '*':mul,
        '/':truediv
    }
    interp = lambda exp: lang_int_interp(exp,env)
    match tuple(exp):
        case ("Const",val):
            return val
        case ("Prim",op,left,right) if op in prim_op:
            left = interp(left)
            right = interp(right)
            prim_op = prim_op[op]
            return prim_op(left,right)
        case ("Prim",'-',exp):
            return - interp(exp)
        case _:
            print('lang_int_interp Unknown Expression: ', tuple(exp))
            raise NotImplementedError

def lang_let_interp(exp,env:Env):
    interp = lang_let_interp
    match tuple(exp):
        case ("Let",var,exp_val,body):
            val = interp(exp_val,env)
            new_env = env.extend_from_vars_vals((var,),(val,))
            return interp(body,new_env)
        case ("Var",var):
            return env.lookup(var)
        case _:
            return lang_int_interp(exp,env)

try:
    interp = lang_let_interp
    exp = Prim('+',(Prim('*',(Const(10),Prim('/',(Const(9),Const(5))))),Const(32)))
    val = interp(exp,Env())
    assert(val == 50)
    
    exp = Let('y',Const(10),
                Prim('-',(Var('y'),)))
    interp(exp,Env())
except NotImplementedError as e:
    print(e)

lang_int_interp Unknown Expression:  ('Var', 'y')



# Solution : OOP - Inheritance

In [5]:
class Lang_Int_Interpreter:
    from operator import sub,add,mul,truediv
    prim_op = {
        '-':sub,
        '+':add,
        '*':mul,
        '/':truediv
    }
    def interp(self,exp,env:Env = None):
        interp = lambda exp: self.interp(exp,env)
        prim_op = Lang_Int_Interpreter.prim_op
        match tuple(exp):
            case ("Const",val):
                return val
            case ("Prim",op,left,right) if op in prim_op:
                left = interp(left)
                right = interp(right)
                return prim_op[op](left,right)
            case ("Prim",'-',exp):
                return - interp(exp)
            case _:
                print('lang_int_interp Unknown Expression: ', tuple(exp))
                raise NotImplementedError

class Lang_Var_Interpreter(Lang_Int_Interpreter):
    def interp(self,exp,env:Env):
        interp = self.interp
        match tuple(exp):
            case ("Let",var,exp_val,body):
                val = interp(exp_val,env)
                new_env = env.extend_from_vars_vals((var,),(val,))
                return interp(body,new_env)
            case ("Var",var):
                return env.lookup(var)
            case _:
                return super(Lang_Var_Interpreter,self).interp(exp,env)


interp = Lang_Var_Interpreter().interp
exp = Prim('+',(Prim('*',(Const(10),Prim('/',(Const(9),Const(5))))),Const(32)))

val = interp(exp,Env())
assert(val == 50)

exp = Let('y',Const(10),
              Prim('-',(Var('y'),)))
interp(exp,Env())

-10

# Solution : OOP - Fowarding

In [9]:
class Lang_Int_Interpreter:
    from operator import sub,add,mul,truediv
    prim_op = {
        '-':sub,
        '+':add,
        '*':mul,
        '/':truediv
    }
    def interp(self,exp,env:Env = None):
        interp = lambda exp: self.interp(exp,env)
        prim_op = Lang_Int_Interpreter.prim_op
        match tuple(exp):
            case ("Const",val):
                return val
            case ("Prim",op,left,right) if op in prim_op:
                left = interp(left)
                right = interp(right)
                return prim_op[op](left,right)
            case ("Prim",'-',exp):
                return - interp(exp)
            case _:
                print('lang_int_interp Unknown Expression: ', tuple(exp))
                raise NotImplementedError

class Lang_Var_Interpreter:
    def interp(self,exp,env:Env):
        interp = self.interp
        match tuple(exp):
            case ("Let",var,exp_val,body):
                val = interp(exp_val,env)
                new_env = env.extend_from_vars_vals((var,),(val,))
                return interp(body,new_env)
            case ("Var",var):
                return env.lookup(var)
            case _:
                return Lang_Int_Interpreter.interp(self,exp,env)


interp = Lang_Var_Interpreter().interp

exp = Prim('+',(Prim('*',(Const(10),Prim('/',(Const(9),Const(5))))),Const(32)))
val = interp(exp,Env())
assert(val == 50)

exp = Let('y',Const(10),
              Prim('-',(Var('y'),)))
interp(exp,Env())

-10

# Pass Interpreter - Workaround

In [8]:
def lang_int_interp(exp,env:Env = None,interp=None):
    from operator import sub,add,mul,truediv
    prim_op = {
        '-':sub,
        '+':add,
        '*':mul,
        '/':truediv
    }
    _interp = lang_int_interp if interp is None else interp
    interp = lambda exp: _interp(exp,env)
    match tuple(exp):
        case ("Const",val):
            return val
        case ("Prim",op,left,right) if op in prim_op:
            left = interp(left)
            right = interp(right)
            prim_op = prim_op[op]
            return prim_op(left,right)
        case ("Prim",'-',exp):
            return - interp(exp)
        case _:
            print('lang_int_interp Unknown Expression: ', tuple(exp))
            raise NotImplementedError

def lang_let_interp(exp,env:Env,interp=None):
    interp = lang_let_interp if interp is None else interp
    match tuple(exp):
        case ("Let",var,exp_val,body):
            val = interp(exp_val,env)
            new_env = env.extend_from_vars_vals((var,),(val,))
            return interp(body,new_env)
        case ("Var",var):
            return env.lookup(var)
        case _:
            return lang_int_interp(exp,env,interp)

interp = lang_let_interp
exp = Prim('+',(Prim('*',(Const(10),Prim('/',(Const(9),Const(5))))),Const(32)))

val = interp(exp,Env())
assert(val == 50)

exp = Let('y',Const(10),
              Prim('-',(Var('y'),)))
interp(exp,Env())

-10